Requirement is on https://github.com/itatsmove/smovechallenge/blob/master/challenges/bookingordering.md
Read json data into a DataFrame

In [1]:
import pandas as pd

In [2]:
Orders = pd.read_json('bookingordering.json')

In [3]:
# add one column to the orignal data set for next step
Orders['processed'] = 0
Orders

,end,id,start,processed
0,1,1,3,0
1,3,2,1,0
2,1,3,3,0
3,2,4,2,0
4,1,5,3,0
5,3,6,2,0
6,3,7,1,0
7,1,8,1,0
8,3,9,3,0
9,2,10,3,0


In [4]:
# build up a table to record total 'start's (drive out) and 'end's (drive in) and the remains (ends - starts) for each location

Locations = pd.DataFrame({'LocationID': [], 'Total_Starts': [], 'Total_Ends': []})

LocationsList = []

for _, oRow in Orders.iterrows():
          
    if oRow['end'] in LocationsList:
        #print (Locations.loc[Locations['LocationID'] == oRow['end']])
        Locations.loc[Locations['LocationID'] == oRow['end'], 'Total_Ends'] += 1
        #print (Locations.loc[Locations['LocationID'] == oRow['end']])
    else:
        Locations = Locations.append({'LocationID': oRow['end'], 'Total_Starts': 0, 'Total_Ends':1}, ignore_index=True)
        LocationsList.append(oRow['end'])
       
    if oRow['start'] in LocationsList:
        Locations.loc[Locations['LocationID'] == oRow['start'], 'Total_Starts'] += 1
    else:
        Locations = Locations.append({'LocationID': oRow['start'], 'Total_Starts': 1, 'Total_Ends':0}, ignore_index=True)
        LocationsList.append(oRow['start'])

Locations = Locations.astype('int32')

remains = abs(Locations.Total_Ends - Locations.Total_Starts)
Locations['remains'] = remains

Locations



,LocationID,Total_Starts,Total_Ends,remains
0,1,6,5,1
1,3,10,8,2
2,2,2,5,3


In [5]:
# Process Start location is same as the end location - mutiple orders
def Process_Order_One_Location(LocationID):
    idList = []
    Count = Orders.loc[(Orders['processed'] == 0) & (Orders['start'] == LocationID) & (Orders['end'] == LocationID), 'id' ].count()
    if Count > 0:
        # process all start == end ==  LocationID
        idList = (Orders.loc[(Orders['processed'] == 0) & (Orders['start'] == LocationID) & (Orders['end'] == LocationID), 'id'].tolist())
        Locations.loc[Locations['LocationID'] == LocationID, 'Total_Starts'] -= Count
        Locations.loc[Locations['LocationID'] == LocationID, 'Total_Ends']   -= Count
        Orders.loc[(Orders['processed'] == 0) & (Orders['start'] == LocationID) & (Orders['end'] == LocationID),'processed' ] = 1 
    return idList

In [6]:
# Process Start location is different from the end location - only first order           
def Process_Order_Two_Locations(StartID ):
    idList = []
    EndID = 0
    if Orders.loc[ (Orders['processed'] == 0) & (Orders['start'] == StartID), 'id' ].count() > 0:
        # find all possible EndID
        EndList = Orders.loc[ (Orders['processed'] == 0) & (Orders['start'] == StartID),'end' ].unique().tolist()
        
        # find the mininmum remains ends
        if Locations.loc[Locations['LocationID'].isin(EndList),'remains'].count() > 0:
            minLocationIndex = Locations.loc[Locations['LocationID'].isin(EndList),'remains'].idxmin()
            EndID = Locations.iloc[ minLocationIndex , 0 ]
            if Orders.loc[(Orders['processed'] == 0) & (Orders['start'] == StartID) & (Orders['end'] == EndID), 'id'].count() > 0:
            # process the first row only
                OrderID = Orders.loc[(Orders['processed'] == 0) & (Orders['start'] == StartID) & (Orders['end'] == EndID) ].iloc[0]['id']
                idList.append(OrderID)
                Orders.loc[Orders['id'] == OrderID, 'processed']  = 1
                Locations.loc[Locations['LocationID'] == StartID, 'Total_Starts'] -= 1
                Locations.loc[Locations['LocationID'] == EndID, 'Total_Ends']     -= 1

                Locations.loc[Locations['LocationID'] == StartID, 'remains'] = abs ( Locations.loc[Locations['LocationID'] == StartID, 'Total_Ends'] - Locations.loc[Locations['LocationID'] == StartID, 'Total_Starts'])
                Locations.loc[Locations['LocationID'] == EndID, 'remains'] = abs ( Locations.loc[Locations['LocationID'] == EndID, 'Total_Ends'] - Locations.loc[Locations['LocationID'] == EndID, 'Total_Starts'])
             
                
    return idList, EndID

In [7]:
Relocations = []  # relocation set


while Locations.loc[Locations['Total_Starts'] > 0,  'remains'].count() > 0: # Loop any 
        
    Relocation = [] # one relocation

    # Start with mininmum remains starts 
    minLocationIndex = Locations.loc[Locations['Total_Starts'] > 0,  'remains'].idxmin()

    StartLocationID = Locations.iloc[ minLocationIndex , 0 ]

    # Find the all start Location ID = End Location ID in Orders
    # add the order id to Relocation list and mark the order as proccessed  
    Relocation.extend(Process_Order_One_Location(StartLocationID))

    # Find the minimum remains End location ID which start location is given
    # add the order id to Relocation list and mark the order as proccessed    
    IDList, EndID = Process_Order_Two_Locations (StartLocationID )
    Relocation.extend(IDList)

    # print (Relocation)

    while EndID > 0:  # repeat the steps as above until no linked location ID can be found 

        StartLocationID = EndID
        Relocation.extend(Process_Order_One_Location(StartLocationID))

        IDList, EndID = Process_Order_Two_Locations (StartLocationID )
        Relocation.extend(IDList)

    Relocations.append(Relocation)
    


In [8]:
# Results and checking
f"Reclocation Count: {len(Relocations)}",  Relocations, 


('Reclocation Count: 3',
 [[8, 13, 2, 9, 11, 18, 1, 7, 3, 14, 5, 17, 10, 4, 6, 12], [15], [16]])

In [9]:
Orders


,end,id,start,processed
0,1,1,3,1
1,3,2,1,1
2,1,3,3,1
3,2,4,2,1
4,1,5,3,1
5,3,6,2,1
6,3,7,1,1
7,1,8,1,1
8,3,9,3,1
9,2,10,3,1


In [10]:
Locations

,LocationID,Total_Starts,Total_Ends,remains
0,1,0,0,0
1,3,0,0,0
2,2,0,0,0
